In [4]:
import json
from PIL import Image
import torch
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
# Create the model (use the same architecture as the trained model)
model = EfficientNet.from_name('efficientnet-b0')  # or the appropriate version

# Load your trained weights
checkpoint = torch.load('../examples/imagenet/model_best.pth.tar')
state_dict = checkpoint['state_dict']

# Remove 'module.' prefix from the keys
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('module.'):
        new_state_dict[k[7:]] = v  # remove 'module.'
    else:
        new_state_dict[k] = v

# Load the modified state dictionary into the model
model.load_state_dict(new_state_dict)

# Preprocess image
tfms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
img = tfms(Image.open('115.jpg')).unsqueeze(0)
print(img.shape)  # torch.Size([1, 3, 224, 224])

# Load ImageNet class names
with open('labels_map.txt') as f:
    labels_map = json.load(f)

num_classes = len(labels_map)
labels_map = [labels_map[str(i)] for i in range(num_classes)]

# Classify
model.eval()
with torch.no_grad():
    outputs = model(img)

# Print predictions
print('-----')
img = Image.open('115.jpg')
plt.imshow(img)
plt.show()

for idx in torch.topk(outputs, k=5).indices.squeeze(0).tolist():
    if idx < num_classes:
        prob = torch.softmax(outputs, dim=1)[0, idx].item()
        print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))
    else:
        print(f'Class index {idx} out of range for available labels.')


torch.Size([1, 3, 224, 224])
-----


NameError: name 'plt' is not defined